In [16]:
import geopandas as gpd
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import rioxarray

In [3]:
flood_data=rioxarray.open_rasterio('bgd_flood_forecast_2017_aug.tif')[0]
population_data=rioxarray.open_rasterio('bgd_flood_forecast_2017_aug.tif')[0]
print(flood_data)
print(flood_data.attrs)

<xarray.DataArray (y: 22485, x: 17303)> Size: 2GB
[389057955 values with dtype=uint32]
Coordinates:
    band         int32 4B 1
  * x            (x) float64 138kB 88.01 88.01 88.01 88.01 ... 92.67 92.67 92.67
  * y            (y) float64 180kB 26.63 26.63 26.63 26.63 ... 20.58 20.58 20.57
    spatial_ref  int32 4B 0
Attributes:
    AREA_OR_POINT:  Area
    _FillValue:     0
    scale_factor:   1.0
    add_offset:     0.0
{'AREA_OR_POINT': 'Area', '_FillValue': 0, 'scale_factor': 1.0, 'add_offset': 0.0}


In [5]:
affected_population=(population_data*(flood_data/1.5))
affected_population=affected_population.where(flood_data>1.5,population_data)
print(affected_population)

In [14]:

districts = gpd.read_file('bgd_adm2.shp')
print(districts)
affected_population_cliped=affected_population.rio.clip(districts.geometry,districts.crs)

    Shape_Leng  Shape_Area     ADM2_EN ADM2_PCODE ADM2_REF ADM2ALT1EN  \
0    12.929816    0.319709    Bagerhat       4001     None       None   
1     5.358126    0.401359   Bandarban       2003     None       None   
2     4.167970    0.117801     Barguna       1004     None       None   
3     7.713625    0.195228     Barisal       1006     None       None   
4    10.091155    0.170812       Bhola       1009     None       None   
..         ...         ...         ...        ...      ...        ...   
59    3.596003    0.221992   Sirajganj       5088     None       None   
60    4.741768    0.330139   Sunamganj       6090     None       None   
61    4.338167    0.305453      Sylhet       6091     None       None   
62    3.644558    0.299525     Tangail       3093     None       None   
63    3.245973    0.163550  Thakurgaon       5594     None       None   

   ADM2ALT2EN     ADM1_EN ADM1_PCODE     ADM0_EN ADM0_PCODE       date  \
0        None      Khulna         40  Bangladesh 

In [ ]:
districts['affected_population']=districts.geometry.apply(lambda x: affected_population_cliped.rio.clip([x],districts.crs).sum().item())

districts_sorted=districts.sort_values('affected_population',ascending=False)

plt.figure(figsize=(10, 6))
plt.bar(districts_sorted['ADM2_EN'],districts_sorted['affected_population'])
plt.ylabel('affected_population')
plt.xlabellabel('Districts')
plt.title('Affected Population by District')
plt.show()


In [ ]:
education_facilities = gpd.read_file('bgd_education_facilities.shp')

affected_facilities = education_facilities[education_facilities.geometry.apply( lambda x: affected_population_clipped.rio.clip([x], education_facilities.crs).max().item() > 1)]

count_affected_facilities = len(affected_facilities)
print('Number of affected educational facilities',count_affected_facilities)